In [1]:
import sys
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.dataset as ds

sys.path.append(str(Path.cwd().parent))
from config import PATH_KIOSK_USER_PATTERNS_FOLDER, PATH_KIOSK_USER_PATTERNS_REPO, PATH_SSD_ADVAN_FOLDER

In [ ]:
import pandas as pd

execfile(Path(PATH_KIOSK_USER_PATTERNS_REPO, "functions/safe_parse_json.py"))


home_panel_files = list((PATH_SSD_ADVAN_FOLDER / "Monthly Patterns" / "Home Panel Summary").rglob('**/*.gz'))
len(home_panel_files)

58

In [5]:
import re

# Extract year from file path using regex (assuming year is a 4-digit number in the path)
home_panel_files_df = pd.DataFrame({
    'file_path': home_panel_files,
    'FILE_NAME': [f.name for f in home_panel_files]
})

pd.options.display.max_colwidth = None  # Show full file paths in the DataFrame
home_panel_files_df.head()

,file_path,FILE_NAME
0,E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_0_0.csv.gz,home-panel-summary_0_0_0.csv.gz
1,E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_1_0.csv.gz,home-panel-summary_0_1_0.csv.gz
2,E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_3_0.csv.gz,home-panel-summary_0_3_0.csv.gz
3,E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_4_0.csv.gz,home-panel-summary_0_4_0.csv.gz
4,E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_5_0.csv.gz,home-panel-summary_0_5_0.csv.gz


In [11]:
output_base = PATH_KIOSK_USER_PATTERNS_FOLDER / "working/processed/kupdat03_advan research home panel summary"


In [ ]:
f = home_panel_files_df['file_path'].iloc[0]
df_table = pd.read_csv(f, 
                              compression='gzip', 
                              engine='python',
                              on_bad_lines='skip')



,CENSUS_BLOCK_GROUP,ISO_COUNTRY_CODE,MON,NUMBER_DEVICES_PRIMARY_DAYTIME,NUMBER_DEVICES_RESIDING,REGION,YEAR


In [17]:
error_list = []


for f in home_panel_files_df['file_path']:
    # Read the gzip-compressed CSV file
    print(f"Processing file: {f}")
    
    try:
        df_table = pd.read_csv(f, 
                              compression='gzip', 
                              engine='python',
                              on_bad_lines='skip')

        # Create a dictionary mapping state abbreviations to full names
        state_abbrev_to_name = {
            'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
            'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
            'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
            'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
            'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
            'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
            'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
            'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
            'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
            'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
            'DC': 'District of Columbia', 'PR': 'Puerto Rico', 'VI': 'Virgin Islands', 'GU': 'Guam',
            'AS': 'American Samoa', 'MP': 'Northern Mariana Islands'
        }
        
        df_usa = (df_table[df_table['ISO_COUNTRY_CODE'] == 'US']
              .rename(columns={'MON': 'MONTH'})
              .assign(FILE_NAME = lambda x: f.name)
              .assign(state_name = lambda x: x['REGION'].map(state_abbrev_to_name)))
        

        
        # Print data summary
        print(f"Found {len(df_usa)} USA records")
        # Save to Parquet with partitioning by YEAR and MONTH
        df_usa.to_parquet(
            output_base,
            index=False,
            partition_cols=["YEAR", "MONTH"]
        )
    except Exception as e:
        print(f"Error processing file {f}: {e}")
        error_list.append(f)
        continue

Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_0_0.csv.gz
Found 219938 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_1_0.csv.gz
Found 219972 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_3_0.csv.gz
Found 220654 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_4_0.csv.gz
Found 439776 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_5_0.csv.gz
Found 217123 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_6_0.csv.gz
Found 219771 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_0_7_0.csv.gz
Found 219737 USA records
Processing file: E:\Advan Research\Monthly Patterns\Home Panel Summary\home-panel-summary_1_0_0.csv.gz
F